In [ ]:
!pip install opencv-python torchvision torchaudio ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 15.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers


In [ ]:
import cv2
import torch
from ultralytics import YOLO
import json

# Load YOLOv8 model
yolo_model = YOLO('yolov8s.pt')

# Open video file
video_path = '/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps)  # Extract one frame per second

# YOLOv8 expects input images to be resized to 640x640 by default
input_size = (640, 640)

detections = []

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        # Resize the frame to the input size expected by YOLOv8
        resized_frame = cv2.resize(frame, input_size)

        # Perform object detection
        results = yolo_model(resized_frame)

        # Extract detection data with threshold of 0.6
        frame_detections = []
        for result in results[0].boxes:
            bbox = result.xyxy[0].tolist()
            confidence = result.conf[0].item()
            class_id = int(result.cls[0].item())

            if confidence >= 0.6:
                frame_detections.append({
                    'bbox': bbox,
                    'confidence': confidence,
                    'class': class_id
                })

        # Store results
        detections.append({
            'frame': int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
            'detections': frame_detections
        })

    frame_count += 1

cap.release()

# Save detections to a file
with open('detections.json', 'w') as f:
    json.dump(detections, f, indent=4)



0: 640x640 (no detections), 16.4ms
Speed: 1.7ms preprocess, 16.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 scissors, 16.5ms
Speed: 3.7ms preprocess, 16.5ms inference, 635.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 scissors, 16.5ms
Speed: 2.1ms preprocess, 16.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 16.6ms
Speed: 1.9ms preprocess, 16.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1 truck, 16.6ms
Speed: 2.0ms preprocess, 16.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 16.6ms
Speed: 2.0ms preprocess, 16.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 16.8ms
Speed: 2.7ms preprocess, 16.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 16.5ms
Speed: 2.8ms preprocess, 16.5ms inference, 1.2ms postprocess per image a

In [ ]:
import cv2
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
import json
from PIL import Image
import io

# Load YOLOv8 medium model
from ultralytics import YOLO
yolo_model = YOLO('yolov8m.pt')

# Define class names (this list should match the class IDs used by your YOLO model)
class_names = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
    'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
    'hair drier', 'toothbrush'
]

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

def generate_description(detections, image):
    # Convert image to a format suitable for BART (e.g., base64 or text description)
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    image_str = buffered.getvalue()

    # Create a description input for BART
    description_input = f"Frame image data: {image_str}\nDetections: {json.dumps(detections)}"

    # Tokenize and generate description
    inputs = tokenizer(description_input, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=150, min_length=30, length_penalty=2.0, num_beams=4)
    description = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return description

# Open video file
video_path = '/content/The Performance-oriented ŠKODA SLAVIA 1.5 L TSI - A Class of its Own (1).mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps)  # Extract 1 frame per second

print(f"Video FPS: {fps}")
print(f"Frame Interval: {frame_interval}")

# Lists to store frames and detections
frame_list = []
detections_list = []

# Initialize sequential frame counter
sequential_frame_count = 1

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break

    # Extract 1 frame per second
    if frame_count % frame_interval == 0:
        frame_list.append(frame)

        # Resize the frame to the input size expected by YOLOv8
        resized_frame = cv2.resize(frame, (640, 640))

        # Perform object detection
        results = yolo_model(resized_frame)

        # Extract detection data with threshold of 0.7
        frame_detections = []
        for result in results[0].boxes:
            bbox = result.xyxy[0].tolist()
            confidence = result.conf[0].item()
            class_id = int(result.cls[0].item())
            class_name = class_names[class_id] if class_id < len(class_names) else 'unknown'

            if confidence >= 0.7:
                frame_detections.append({
                    'bbox': bbox,
                    'confidence': confidence,
                    'class': class_name
                })

        # Convert frame to PIL Image
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Generate description
        #description = generate_description(frame_detections, image)
        print(f"Frame {sequential_frame_count}")

        # Store detections with sequential frame numbering
        detections_list.append({
            'frame': sequential_frame_count,
            'detections': frame_detections,
        })

        # Increment the sequential frame counter
        sequential_frame_count += 1

    frame_count += 1

cap.release()

# Save detections to JSON file
with open('detections.json', 'w') as f:
    json.dump(detections_list, f, indent=4)

print("Detections saved to detections.json")


Video FPS: 25.0
Frame Interval: 25

0: 640x640 (no detections), 37.3ms
Speed: 2.2ms preprocess, 37.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Frame 1

0: 640x640 1 scissors, 37.4ms
Speed: 2.8ms preprocess, 37.4ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)
Frame 2

0: 640x640 1 scissors, 37.4ms
Speed: 2.3ms preprocess, 37.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Frame 3

0: 640x640 1 car, 37.3ms
Speed: 3.5ms preprocess, 37.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Frame 4

0: 640x640 1 car, 37.3ms
Speed: 3.5ms preprocess, 37.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Frame 5

0: 640x640 1 truck, 37.3ms
Speed: 3.1ms preprocess, 37.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Frame 6

0: 640x640 1 person, 1 car, 37.3ms
Speed: 2.6ms preprocess, 37.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Frame 7

0: 640x640 1 pe

In [ ]:
import json
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to('cuda')

def generate_description(detections_list):
    # Format all detections from the video
    formatted_detections = []
    for entry in detections_list:
        frame = entry.get('frame')
        detections = entry.get('detections', [])
        for detection in detections:
            if 'bbox' in detection and 'confidence' in detection and 'class' in detection:
                formatted_detections.append({
                    'frame': frame,
                    'bbox': detection['bbox'],
                    'confidence': detection['confidence'],
                    'class': detection['class']
                })

    # Create a single input for the entire video
    description_input = f"Video detections: {json.dumps(formatted_detections)}"

    # Tokenize and generate a single description for the whole video
    inputs = tokenizer(description_input, return_tensors="pt", max_length=1024, truncation=True).to('cuda')
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4)
    description = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return description

def process_detections_file(file_path):
    # Load detections from JSON file
    with open(file_path, 'r') as f:
        detections_list = json.load(f)

    # Generate a single description for the whole video
    description = generate_description(detections_list)

    return description

# Example usage
file_path = 'detections.json'
video_description = process_detections_file(file_path)
print(f"Video Description: {video_description}")




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Video Description: Video detections: [{"frame": 4, "bbox": [36.71832275390625, 256.89569091796875, 455.08587646484375, 429.8542113304138184, "class": "car"}, {"frame": 5, "featured": [0.class62,125.bbox], 2352822265625, 282.83953857421875, 334.7838249206543, 146.6851801664775], "confidence": 0. {"frame: 6, "box": 546.712418258190155, 192.76895141601562, 444


In [ ]:
import json
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to('cuda')

def clean_input(input_text):
    # Remove unwanted characters or symbols
    allowed_chars = ''.join(chr(i) for i in range(32, 127))  # ASCII characters from space to tilde (~)
    cleaned_text = ''.join(c for c in input_text if c in allowed_chars)
    return cleaned_text

def generate_description(detections_summary):
    # Ensure the input is cleaned before feeding it to the BART model
    description_input = f"The following objects were detected in the video:\n{clean_input(detections_summary)}"

    # Tokenize and generate description
    inputs = tokenizer(description_input, return_tensors="pt", max_length=512, truncation=True).to('cuda')
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4)
    description = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return description

def process_detections_file(file_path):
    # Load detections from JSON file
    with open(file_path, 'r') as f:
        detections_list = json.load(f)

    # Collect detected objects along with their frame numbers
    frame_objects = {}
    for entry in detections_list:
        frame = entry.get('frame')
        detections = entry.get('detections', [])
        objects_in_frame = []

        for detection in detections:
            if 'class' in detection:
                class_name = detection['class']
                objects_in_frame.append(class_name)

        # Save objects detected in this frame
        if objects_in_frame:
            frame_objects[frame] = objects_in_frame

    # Create a formatted summary
    detections_summary = ""
    for frame, objects in frame_objects.items():
        objects_str = ', '.join(objects)
        detections_summary += f"Frame {frame}: {objects_str}\n"

    # Generate a single description for the whole video
    description = generate_description(detections_summary)

    return description

# Example usage
file_path = 'detections.json'
video_description = process_detections_file(file_path)
print(f"Video Description: {video_description}")


Video Description: The following objects were detected in the video:Frame 1: car, personFrame 2: personFrame 3: person, carFrame 4: car frame 5: carFrame 6: person frame 7: person Frame 8: car Frame 9: car and personFrame 9: vehicleFrame 10: carFrames 11: personFrames 12: carframes 13: car framesFrame 14: vaseFrame 15: person.Frame 16: car.Frame 17: car


In [ ]:
import json
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to('cuda')

def clean_input(input_text):
    # Remove unwanted characters or symbols
    allowed_chars = ''.join(chr(i) for i in range(32, 127))  # ASCII characters from space to tilde (~)
    cleaned_text = ''.join(c for c in input_text if c in allowed_chars)
    return cleaned_text

def generate_description(detections_summary):
    # Ensure the input is cleaned before feeding it to the BART model
    description_input = f"The following objects and their positions were detected in the video:\n{clean_input(detections_summary)}"

    # Tokenize and generate description
    inputs = tokenizer(description_input, return_tensors="pt", max_length=512, truncation=True).to('cuda')
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4)
    description = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return description

def process_detections_file(file_path):
    # Load detections from JSON file
    with open(file_path, 'r') as f:
        detections_list = json.load(f)

    # Sort the detection list by frame number to ensure sequential order
    detections_list = sorted(detections_list, key=lambda x: x['frame'])

    # Collect detected objects along with their frame numbers and positions (bounding boxes)
    frame_objects = {}
    for entry in detections_list:
        frame = entry.get('frame')
        detections = entry.get('detections', [])
        objects_in_frame = []

        for detection in detections:
            if 'class' in detection and 'bbox' in detection:
                class_name = detection['class']
                bbox = detection['bbox']
                # Convert bounding box to a readable format (e.g., top-left and bottom-right)
                if len(bbox) == 4:  # Only process if the bounding box has the correct format
                    bbox_str = f"({bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f})"
                    objects_in_frame.append(f"{class_name} at {bbox_str}")

        # Save objects detected in this frame
        if objects_in_frame:
            frame_objects[frame] = objects_in_frame

    # Create a formatted summary including objects and their bounding box positions
    detections_summary = ""
    for frame, objects in frame_objects.items():
        objects_str = '; '.join(objects)
        detections_summary += f"Frame {frame}: {objects_str}\n"

    # Generate a single description for the whole video
    description = generate_description(detections_summary)

    return description

# Example usage
file_path = 'detections.json'
video_description = process_detections_file(file_path)
print(f"Video Description: {video_description}")


Video Description: The following objects and their positions were detected in the video:Frame 4: car at (36.7, 84.9, 85.5, 83.6, 113.1, 135.7)Frame 6: person at (94.2, 139.8, 146.3, 150.7; car at 144.4, 157.4)Frame 18: vase at (204.2)Frame 23: car (169.1)Frame 32: person (168.1); car at 18.1; car (18.0); person at 1.7); car (0.0)Frame 8: car and truck at (304.0, 336.8; car(s)


In [ ]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801360 sha256=d1f70006406e18af96b18ed91a6c90b55112e42887470326f7bcd42552de614c
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper


In [ ]:
import whisper

def transcribe_audio(audio_path, output_file):
    model = whisper.load_model("medium")
    result = model.transcribe(audio_path, language="en", temperature=0.6, verbose=True)

    with open(output_file, 'w') as f:
        for segment in result['segments']:
            start_time = segment['start']
            end_time = segment['end']
            text = segment['text']
            f.write(f"[{start_time:.2f} - {end_time:.2f}] {text}\n")

# Example usage
audio_path = "/content/audio file.m4a"
output_file = "/content/transcription_with_timestamps.txt"
transcribe_audio(audio_path, output_file)


100%|█████████████████████████████████████| 1.42G/1.42G [00:20<00:00, 76.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

[00:00.000 --> 00:04.200]  There are many routes one can take in life.
[00:04.200 --> 00:07.640]  Sometimes we take a beautiful
[00:07.640 --> 00:10.920]  but measured journey.
[00:10.920 --> 00:14.920]  But when life is full of choices...
[00:14.920 --> 00:21.920]  sometimes you just let the heart take over.
[00:30.000 --> 00:45.920]  When exhilaration drives you to aim higher.
[00:45.920 --> 00:51.360]  When emotion overtakes all else.
[00:51.360 --> 00:52.880]  ŠKODA SLAVIA 1.5 TSI


In [ ]:
import json
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Detect if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bart_model = bart_model.to(device)

def clean_input(input_text):
    # Remove unwanted characters or symbols
    allowed_chars = ''.join(chr(i) for i in range(32, 127))  # ASCII characters from space to tilde (~)
    cleaned_text = ''.join(c for c in input_text if c in allowed_chars)
    return cleaned_text

def generate_description(detections_summary, transcribed_text):
    # Combine detections summary and transcribed text
    description_input = (f"Please generate a detailed textual summary of the video based on Detected Objects and Transcribed Audio\n\n"
                         f"{clean_input(detections_summary)}\n\n"
                         f"{clean_input(transcribed_text)}\n\n")

    # Debug: Print the final input to check formatting
    print("Description Input:\n", description_input)

    # Tokenize and generate description
    inputs = tokenizer(description_input, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=700, min_length=100, length_penalty=2.0, num_beams=4)
    description = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return description

def process_detections_file(detections_file_path, transcribed_audio_file_path):
    # Load detections from JSON file
    with open(detections_file_path, 'r') as f:
        detections_list = json.load(f)

    # Load transcribed audio text from TXT file
    with open(transcribed_audio_file_path, 'r') as f:
        transcribed_text = f.read()

    # Sort the detection list by frame number to ensure sequential order
    detections_list = sorted(detections_list, key=lambda x: x['frame'])

    # Collect detected objects along with their frame numbers and positions (bounding boxes)
    frame_objects = {}
    for entry in detections_list:
        frame = entry.get('frame')
        detections = entry.get('detections', [])
        objects_in_frame = []

        for detection in detections:
            if 'class' in detection and 'bbox' in detection:
                class_name = detection['class']
                bbox = detection['bbox']
                # Convert bounding box to a readable format (e.g., top-left and bottom-right)
                if len(bbox) == 4:  # Only process if the bounding box has the correct format
                    bbox_str = f"({bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f})"
                    objects_in_frame.append(f"{class_name} at {bbox_str}")

        # Save objects detected in this frame
        if objects_in_frame:
            frame_objects[frame] = objects_in_frame

    # Create a formatted summary including objects and their bounding box positions
    detections_summary = ""
    for frame, objects in frame_objects.items():
        objects_str = '; '.join(objects)
        detections_summary += f"Frame {frame}: {objects_str}\n"

    # Debug: Print formatted summaries to check if they are correct
    print("Detections Summary:\n", detections_summary)
    print("Transcribed Text:\n", transcribed_text)

    # Generate a single description for the whole video including transcribed audio
    description = generate_description(detections_summary, transcribed_text)

    return description

# Example usage
detections_file_path = 'detections.json'
transcribed_audio_file_path = 'transcription_with_timestamps.txt'
video_description = process_detections_file(detections_file_path, transcribed_audio_file_path)
print(f"Video Description: {video_description}")


Detections Summary:
 Frame 4: car at (36.8, 146.4, 157.4, 267.7)
Frame 5: car at (94.7, 150.7, 220.0, 282.7)
Frame 7: car at (287.3, 192.9, 455.1, 430.0)
Frame 8: car at (304.8, 221.8, 444.2, 418.0); person at (377.3, 219.3, 382.7, 234.5)
Frame 9: car at (289.4, 238.6, 423.5, 406.1); car at (169.8, 342.6, 224.5, 405.6)
Frame 10: car at (174.7, 327.7, 256.7, 411.2)
Frame 11: car at (192.3, 296.1, 337.0, 422.1)
Frame 16: car at (55.4, 239.0, 460.2, 479.5); car at (443.8, 334.5, 562.1, 439.4)
Frame 18: vase at (204.5, 83.4, 402.9, 397.4)
Frame 19: person at (156.9, 83.8, 441.7, 551.2)
Frame 24: person at (166.9, 85.7, 637.7, 546.1)
Frame 32: person at (267.2, 139.9, 569.8, 553.1)
Frame 36: car at (159.4, 235.8, 603.2, 500.9)
Frame 38: car at (206.2, 193.8, 526.9, 505.6)
Frame 45: truck at (189.5, 255.7, 251.9, 321.5)
Frame 48: car at (259.1, 135.0, 336.2, 196.0)
Frame 50: car at (18.6, 113.7, 441.9, 370.4)
Frame 51: car at (0.7, 84.5, 160.8, 530.6)
Frame 53: car at (216.6, 279.3, 393.8, 3

In [ ]:
input = """

""" + str()

In [ ]:
import json
from transformers import BartTokenizer, BartForConditionalGeneration
import torch

# Load BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Detect if GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bart_model = bart_model.to(device)

def clean_input(input_text):
    # Remove unwanted characters or symbols
    allowed_chars = ''.join(chr(i) for i in range(32, 127))  # ASCII characters from space to tilde (~)
    cleaned_text = ''.join(c for c in input_text if c in allowed_chars)
    return cleaned_text

def generate_description(detections_summary):
    # Combine detections summary and transcribed text
    description_input = (f"""In this frame number are given that are extracted from the video and its detections are given and then bounding boxes are given based on detections. Predict what might be happening in the video.{detections_summary}""")

    # Debug: Print the final input to check formatting
    print("Description Input:\n", description_input)

    # Tokenize and generate description
    inputs = tokenizer(description_input, return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=700, min_length=100, length_penalty=2.0, num_beams=4)
    description = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return description

def process_detections_file(detections_file_path):
    # Load detections from JSON file
    with open(detections_file_path, 'r') as f:
        detections_list = json.load(f)



    # Sort the detection list by frame number to ensure sequential order
    detections_list = sorted(detections_list, key=lambda x: x['frame'])

    # Collect detected objects along with their frame numbers and positions (bounding boxes)
    frame_objects = {}
    for entry in detections_list:
        frame = entry.get('frame')
        detections = entry.get('detections', [])
        objects_in_frame = []

        for detection in detections:
            if 'class' in detection and 'bbox' in detection:
                class_name = detection['class']
                bbox = detection['bbox']
                # Convert bounding box to a readable format (e.g., top-left and bottom-right)
                if len(bbox) == 4:  # Only process if the bounding box has the correct format
                    bbox_str = f"({bbox[0]:.1f}, {bbox[1]:.1f}, {bbox[2]:.1f}, {bbox[3]:.1f})"
                    objects_in_frame.append(f"{class_name} at {bbox_str}")

        # Save objects detected in this frame
        if objects_in_frame:
            frame_objects[frame] = objects_in_frame

    # Create a formatted summary including objects and their bounding box positions
    detections_summary = ""
    for frame, objects in frame_objects.items():
        objects_str = '; '.join(objects)
        detections_summary += f"Frame {frame}: {objects_str}\n"

    # Debug: Print formatted summaries to check if they are correct
    print("Detections Summary:\n", detections_summary)

    # Generate a single description for the whole video including transcribed audio
    description = generate_description(detections_summary)

    return description

# Example usage
detections_file_path = 'detections.json'
video_description = process_detections_file(detections_file_path)
print(f"Video Description: {video_description}")
